# Erasmus Data Cleaning
This notebook describe the steps I took to clean the Erasmus Data. In the end the data is saved as a pickle file which is already provided in the project. Therefor it is not necessary to run this notebook.

First I created a file with all the available data.

In [1]:
import pandas as pd
import os 
import warnings
import pycountry
import numpy as np
warnings.filterwarnings('ignore')


data_files = [f for f in os.listdir('Erasmus_Data') if (f.endswith('.csv') and 'mobilities eligible finalised started in')]
data = pd.DataFrame()

for file in data_files:
    year_data = pd.read_csv('Erasmus_Data/'+ file, delimiter = ';')
    data = pd.concat([data, year_data], axis=0, ignore_index=True)

data.to_pickle('Erasmus_Data/data_complete.pkl')
print(f"The shape of the data to clean is: {data.shape}")
print(f"The columns in the original dataset are: {data.columns}")
data.head()

The shape of the data to clean is: (3626893, 24)
The columns in the original dataset are: Index(['Project Reference', 'Academic Year', 'Mobility Start Month',
       'Mobility End Month', 'Mobility Duration', 'Activity (mob)',
       'Field of Education', 'Participant Nationality', 'Education Level',
       'Participant Gender', 'Participant Profile', 'Special Needs',
       'Fewer Opportunities', 'GroupLeader', 'Participant Age',
       'Sending Country Code', 'Sending City', 'Sending Organization',
       'Sending Organisation Erasmus Code', 'Receiving Country Code',
       'Receiving City', 'Receiving Organization',
       'Receiving Organisation Erasmus Code', 'Participants'],
      dtype='object')


,Project Reference,Academic Year,Mobility Start Month,Mobility End Month,Mobility Duration,Activity (mob),Field of Education,Participant Nationality,Education Level,Participant Gender,...,Participant Age,Sending Country Code,Sending City,Sending Organization,Sending Organisation Erasmus Code,Receiving Country Code,Receiving City,Receiving Organization,Receiving Organisation Erasmus Code,Participants
0,2014-1-AT01-KA101-000059,2014-2015,2014-07,2014-07,12.0,Staff training abroad,Language acquisition,AT,??? - ? Unknown ?,Female,...,42,AT,Wels,BRG Wels Wallererstraße,-,ES,Malaga,Escuela de Idiomas Nerja,-,1
1,2014-1-AT01-KA101-000059,2014-2015,2014-07,2014-08,7.0,Staff training abroad,Language acquisition,AT,??? - ? Unknown ?,Female,...,25,AT,Wels,BRG Wels Wallererstraße,-,UK,Canterbury,Pilgrims LTD,-,1
2,2014-1-AT01-KA101-000059,2014-2015,2014-07,2014-08,12.0,Staff training abroad,Language acquisition,AT,??? - ? Unknown ?,Female,...,51,AT,Wels,BRG Wels Wallererstraße,-,UK,London,St. Giles International,-,1
3,2014-1-AT01-KA101-000059,2014-2015,2014-08,2014-08,12.0,Staff training abroad,Language acquisition,CZ,??? - ? Unknown ?,Female,...,34,AT,Wels,BRG Wels Wallererstraße,-,UK,London,EF Language Center,-,1
4,2014-1-AT01-KA101-000060,2014-2015,2014-07,2014-07,12.0,Staff training abroad,Teacher training with subject specialization,AT,??? - ? Unknown ?,Female,...,53,AT,Schwarzach,Volksschule Schwarzach,-,IE,Dublin,Centre of English Studies,-,1


Than I dropped the columns without relevent information.

In [2]:
data = data.drop(['Project Reference', 'Academic Year', 'Participant Profile',
                  'GroupLeader', 'Sending Organisation Erasmus Code',
                  'Receiving Organisation Erasmus Code'], axis=1)


Checking and droping NaNs, since they are a not a lot of them compared to the rest of the data.

In [3]:
data.isna().sum()
data.dropna(inplace=True)

Checking for entries without Nationality

In [4]:
print(f"There are {data[data['Participant Nationality'] == '-'].shape[0]} rows without Nationality")
data = data.drop(data[data['Participant Nationality'] == '-'].index, axis=0)

There are 2019 rows without Nationality


Renaming some of the columns

In [5]:
data = data.drop(data[data['Participant Nationality'] == '-'].index, axis=0)
data = data.rename(columns={'Sending Country Code': 'Sending Country', 'Receiving Country Code': 'Receiving Country',
                            'Activity (mob)': 'Activity'})

Replacing the country code with the country name.

In [6]:
country_dict = {c.alpha_2: c.name for c in pycountry.countries}
country_dict.update({'XK': 'Kosovo', 'EL': 'Greece', 'UK': 'United Kingdom', "TP": "East Timor", "AN": "Netherlands Antilles",
                     "AB": "Albania", "CP": "Clipperton Island"})
data[['Participant Nationality',
      'Sending Country',
      'Receiving Country']] = data[['Participant Nationality',
                                    'Sending Country',
                                    'Receiving Country']].applymap(lambda x: country_dict.get(x, x))

Than I repeated the entries where multiple person took part in one project.

In [7]:
data = data.loc[data.index.repeat(data['Participants'])].reset_index(drop=True)
data.shape

(4199400, 18)

Renaming some of the entries.

In [8]:
data['Activity'] = data['Activity'].replace({"Advance Planning Visit – EVS": "Advance Planning Visit - EVS",
                                             "Training/teaching assignments abroad": "Teaching/training assignments abroad"})
data['Field of Education'] = data['Field of Education'].replace(
    "? Unknown ?", "Unknown")

data['Education Level'] = data['Education Level']\
    .replace({"??? - ? Unknown ?": "Unknown",
              "ISCED-2 - Lower secondary education": "Lower Secondary Education",
              "ISCED-3 - Upper secondary education": "Upper Secondary Education",
              "ISCED-4 - Post-secondary non-tertiary education": "Post-secondary non-Tertiary Education",
              "ISCED-5 - Short-cycle within the first cycle / Short-cycle tertiary education (EQF-5)": "Short-cycle within the first cycle / Short-cycle tertiary education",
              "ISCED-6 - First cycle / Bachelor’s or equivalent level (EQF-6)": "First cycle / Bachelor’s or equivalent level",
              "ISCED-7 - Second cycle / Master’s or equivalent level (EQF-7)": "Second cycle / Master’s or equivalent level",
              "ISCED-8 - Third cycle / Doctoral or equivalent level (EQF-8)": "Third cycle / Doctoral or equivalent level",
              "ISCED-9 - Not elsewhere classified": "Not elsewhere classified",
              })

Cleaning the age column. Only values between 5 and 90 are allowed.

In [9]:
def clean_age(x):
    try:
        x = int(x)
    except:
        x = np.nan

    if x < 5 or x > 90:
        x = np.nan
    return x

print(f"These are the unique values in the age column before cleaning: {data['Participant Age'].unique()}")
data['Participant Age'] = data['Participant Age'].apply(lambda x: clean_age(x))

These are the unique values in the age column before cleaning: ['42' '25' '51' '34' '53' '60' '44' '46' '50' '48' '49' '45' '26' '54'
 '30' '40' '43' '31' '33' '47' '52' '62' '55' '56' '57' '35' '27' '36'
 '59' '29' '38' '-1' '39' '32' '58' '28' '61' '-2' '37' '16' '17' '64'
 '18' '19' '20' '21' '63' '24' '22' '15' '23' '0' '65' '-' '41' '6' '7'
 '14' '70' '13' '12' '66' '67' '823' '114' '1' '11' '109' '941' '953'
 '125' '68' '1922' '1929' '1923' '1925' '1928' '69' '84' '91' '917' '73'
 '71' '75' '72' '4' '-184' '708' '814' '-35' '-31' '1016' '962' '8' '822'
 '76' '9' '950' '1030' '957' '928' '10' '1815' '817' '-7939' '79' '1019'
 '-5' '92' '81' '820' '821' '-186' '74' '5' '-3' '82' '-1185' '940' '1060'
 '1057' '77' '-29' '112' '134' '95' '2' '929' '943' '-7171' '-2184' '938'
 '924' 22 26 20 21 40 31 24 30 29 51 25 18 28 19 -2 23 -1 13 14 15 12 50
 56 17 16 34 47 33 27 48 0 39 55 37 35 49 45 54 52 38 32 70 44 36 902 46
 67 41 43 42 57 59 60 53 68 6 65 61 62 58 63 69 7 64 11 10 114 132 

Changing the starting and ending month to datetime format.

In [10]:
data['Mobility Start Month'] = pd.to_datetime(data['Mobility Start Month'])
data['Mobility End Month'] = pd.to_datetime(data['Mobility End Month'])

Replacing entries with booleans and pickle the dataframe.

In [11]:
data['Special Needs'] = data['Special Needs'].replace({'No':0, 'Yes':1})
data['Fewer Opportunities'] = data['Fewer Opportunities'].replace({'No':0, 'Yes':1})

data.to_pickle('Erasmus_Data/data_clean.pkl')

For the recommendation engine I doped some additional columns and filled the missing ages with means. Although there are quite a few missing ages it is still just around 10% of the entries and I did not want to drop this column

In [12]:
print(f"The data['Participant Age'].isna().sum())

SyntaxError: EOL while scanning string literal (204606135.py, line 1)

In [ ]:
mean_age = data['Participant Age'].mean()
data['Participant Age'].fillna(mean_age, inplace=True)
data.to_pickle('Erasmus_Data/model_data.pkl')